In [5]:
import pandas as pd
from scipy.sparse import hstack
import joblib

# Define the MedicalPredictor class
class MedicalPredictor:
    def __init__(self, model_file="medical_predictor_model (1).pkl"):
        self.model_file = model_file
        self.load_model()

    def load_model(self):
        self.encoder, self.tfidf_vectorizer, self.rf_icd10, self.rf_treatment = joblib.load(self.model_file)

    def predict_input(self, input_tuple):
        input_data = pd.DataFrame([input_tuple], columns=['Problem Summary', 'Problem Category', 'Psychological Category'])

        # Transform input data
        input_encoded = self.encoder.transform(input_data[['Problem Category', 'Psychological Category']])
        input_summary = self.tfidf_vectorizer.transform(input_data['Problem Summary'])
        input_combined = hstack((input_encoded, input_summary))

        # Make predictions
        icd10_prediction = self.rf_icd10.predict(input_combined)[0]
        treatment_prediction = self.rf_treatment.predict(input_combined)[0]

        return icd10_prediction, treatment_prediction

# Example usage
predictor = MedicalPredictor(model_file="medical_predictor_model (1).pkl")

input_tuple = ('Obsessive-Compulsive Disorder','Obsessive-Compulsive Disorder','Anxiety')
icd10_pred, treatment_pred = predictor.predict_input(input_tuple)

print("Predicted ICD-10:", icd10_pred)
print("Predicted Treatment Provided:", treatment_pred)


Predicted ICD-10: F42.2
Predicted Treatment Provided: Cognitive Therapy


In [7]:
def train_or_load_model(training_data):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    X_train = tfidf_vectorizer.fit_transform(training_data['Problem Description'])
    y_train = training_data[['Problem Summary', 'Problem Category', 'Psychological Category', 'ICD-10']]

    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    multi_output_classifier = MultiOutputClassifier(classifier)

    pipeline = Pipeline([
        ('tfidf', tfidf_vectorizer),
        ('clf', multi_output_classifier)
    ])

    pipeline.fit(training_data['Problem Description'], y_train)

    return pipeline

In [12]:
def check_for_new_files_and_update_model(folder_path):
    latest_file = max([os.path.join(folder_path, f) for f in os.listdir(folder_path)], key=os.path.getctime)
    print("Latest file:", latest_file)
    training_data = pd.read_csv(latest_file)

    if os.path.exists('/Users/shubhamsaxena/PycharmProjects/noideaProject/ModelState/model.pkl'):
        with open('model.pkl', 'rb') as model_file:
            pipeline = pickle.load(model_file)
    else:
        pipeline = train_or_load_model(training_data)

    pipeline = train_or_load_model(training_data)

    with open('/Users/shubhamsaxena/PycharmProjects/noideaProject/ModelState/model.pkl', 'wb') as model_file:
        pickle.dump(pipeline, model_file)

    return pipeline

In [17]:
with open('model (2).pkl', 'rb') as model_file:
    pipeline = pickle.load(model_file)
def make_predictions_on_incoming_dataset(incoming_dataset, pipeline):
    incoming_data = pd.DataFrame([incoming_dataset])
    incoming_data['Problem Description'] = incoming_data['problem_description'].apply(preprocess_text)
    incoming_data.drop(columns=['problem_description'], inplace=True)

    predictions = pipeline.predict(incoming_data['Problem Description'])
    return predictions

ValueError: node array from the pickle has an incompatible dtype:
- expected: [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]
- got     : {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}